## Defunciones por suicidio por entidad federativa y grupos de edad según sexo, serie anual de 2010 a 2020

Los datos se obtuvieron del sitio web https://www.inegi.org.mx/app/tabulados/interactivos/?pxq=Salud_Mental_06_05d86884-2a0a-40ae-b810-4f6c966e0187&idrt=148&opc=t el 10 de Abril del 2022.

Nota: Solo se pasaron los datos de 10-14 años al directorio de data_clean, sin embargo el EDA se realizo tambien
para el grupo de edad de 15-19 años


In [2]:
#Importando librerias necesarias
import pandas as pd
import pathlib

#Todo lo relacionado a graficar
import plotly.express as px


In [3]:
#Definiendo el Path del archivo
path = pathlib.Path('..', 'data_dir/Mental_06.xlsx')

In [4]:
#Leyendo los datos
data = pd.read_excel(path, header = 3,  skipfooter=10)

In [5]:
#Confirmando que los datos se leyeron correctamente
data

,Entidad federativa,Grupos quinquenales de edad,2010,2010.1,2010.2,2010.3,2011,2011.1,2011.2,2011.3,...,2018.2,2018.3,2019,2019.1,2019.2,2019.3,2020,2020.1,2020.2,2020.3
0,Entidad federativa,Grupos quinquenales de edad,Total,Hombres,Mujeres,No especificado,Total,Hombres,Mujeres,No especificado,...,Mujeres,No especificado,Total,Hombres,Mujeres,No especificado,Total,Hombres,Mujeres,No especificado
1,Sonora,10 a 14 años,6,0,6,0,0,0,0,0,...,1,0,6,6,0,0,2,1,1,0
2,Sonora,15 a 19 años,18,12,6,0,26,19,7,0,...,7,0,20,15,5,0,25,22,3,0


### Limpieza de Datos

In [6]:
# Transponemos el dataset para que el año se encuentre en el indice y renombramos las columnas
sonora_data = data.T.copy()
sonora_data.columns = ['sexo', 'suicidios_edad_10_14', 'suicidios_edad_15_19']
sonora_data.head()

,sexo,suicidios_edad_10_14,suicidios_edad_15_19
Entidad federativa,Entidad federativa,Sonora,Sonora
Grupos quinquenales de edad,Grupos quinquenales de edad,10 a 14 años,15 a 19 años
2010,Total,6,18
2010.1,Hombres,0,12
2010.2,Mujeres,6,6


In [7]:
# Confirmamos que no haya valores nulos 
sonora_data.isnull().sum()

sexo                    0
suicidios_edad_10_14    0
suicidios_edad_15_19    0
dtype: int64

In [8]:
# Removemos los primeros dos registros, ya que sabemos que los datos son de Sonora y los grupos quinquenales se
# encuentran en el nombre de la columna
sonora_data = sonora_data.iloc[2:, :].copy()
sonora_data.head()

,sexo,suicidios_edad_10_14,suicidios_edad_15_19
2010,Total,6,18
2010.1,Hombres,0,12
2010.2,Mujeres,6,6
2010.3,No especificado,0,0
2011,Total,0,26


In [9]:
# Renombramos el indice
sonora_data.index.names = ['fecha']
sonora_data.head()


,sexo,suicidios_edad_10_14,suicidios_edad_15_19
fecha,,,
2010,Total,6,18
2010.1,Hombres,0,12
2010.2,Mujeres,6,6
2010.3,No especificado,0,0
2011,Total,0,26


In [10]:
# Removeremos el punto decimal de el año (en aquellos que lo tengan), ya que pandas se lo agrega a la hora de leer el excel
sonora_data = sonora_data.reset_index()
sonora_data['fecha'] = sonora_data['fecha'].apply(lambda x: x[:4])
sonora_data.set_index('fecha', inplace=True)
sonora_data.head()

,sexo,suicidios_edad_10_14,suicidios_edad_15_19
fecha,,,
2010,Total,6,18
2010,Hombres,0,12
2010,Mujeres,6,6
2010,No especificado,0,0
2011,Total,0,26


In [11]:
# Confirmamos que los tipos de datos son correctos
sonora_data[['suicidios_edad_10_14', 'suicidios_edad_15_19']] = sonora_data[['suicidios_edad_10_14', 'suicidios_edad_15_19']].astype('int64')
sonora_data.dtypes

sexo                    object
suicidios_edad_10_14     int64
suicidios_edad_15_19     int64
dtype: object

In [12]:
sonora_data.head()

,sexo,suicidios_edad_10_14,suicidios_edad_15_19
fecha,,,
2010,Total,6,18
2010,Hombres,0,12
2010,Mujeres,6,6
2010,No especificado,0,0
2011,Total,0,26


In [13]:
#Analizaremos los casos donde el Sexo no es especificado, podemos ver que no hay ningun caso donde el sexo no sea especificado
#Por lo que removeremos esos registros
sonora_data[sonora_data['sexo'] == 'No especificado']

,sexo,suicidios_edad_10_14,suicidios_edad_15_19
fecha,,,
2010,No especificado,0,0
2011,No especificado,0,0
2012,No especificado,0,0
2013,No especificado,0,0
2014,No especificado,0,0
2015,No especificado,0,0
2016,No especificado,0,0
2017,No especificado,0,0
2018,No especificado,0,0


In [14]:
sonora_data = sonora_data[sonora_data['sexo'] != 'No especificado'].copy()

In [15]:
# Transformamos los datos en formato Tidy para facilitar su uso
sonora_data_tidy = pd.melt(sonora_data, id_vars=['sexo'], value_vars=['suicidios_edad_10_14', 'suicidios_edad_15_19'], ignore_index=False,
                value_name = 'suicidios', var_name= 'grupo_edad')

In [16]:
sonora_data_tidy = sonora_data_tidy.reset_index()
sonora_data_tidy['fecha'] = sonora_data_tidy['fecha'].astype('int64')

In [17]:
sonora_data_tidy

,fecha,sexo,grupo_edad,suicidios
0,2010,Total,suicidios_edad_10_14,6
1,2010,Hombres,suicidios_edad_10_14,0
2,2010,Mujeres,suicidios_edad_10_14,6
3,2011,Total,suicidios_edad_10_14,0
4,2011,Hombres,suicidios_edad_10_14,0
...,...,...,...,...
61,2019,Hombres,suicidios_edad_15_19,15
62,2019,Mujeres,suicidios_edad_15_19,5
63,2020,Total,suicidios_edad_15_19,25
64,2020,Hombres,suicidios_edad_15_19,22


### Análisis de datos exploratorio

#### Box plots

In [18]:
fig = px.box(sonora_data_tidy, x="sexo", y="suicidios", 
             title='Distribucion de número de Suicidios de menores en Sonora por Sexo (2010-2020)',
                labels={'suicidios' : 'Numero de Suicidios',
                        'sexo': 'Sexo'})

fig.show()


In [19]:
fig = px.box(sonora_data_tidy, x="sexo", y="suicidios", 
             title='Distribucion de número de Suicidios de menores en Sonora por Grupo de Edad (2010-2020)',
                labels={'suicidios' : 'Numero de Suicidios',
                        'grupo_edad': 'Grupo de Edad'})

fig.show()


In [20]:
fig = px.box(sonora_data_tidy, x="sexo", y="suicidios", color='grupo_edad',
             title='Distribucion de número de Suicidios de menores en Sonora por Grupo de Edad y Sexo (2010-2020)',
                labels={'suicidios' : 'Numero de Suicidios',
                        'grupo_edad': 'Grupo de Edad',
                        'sexo':'Sexo'})
newnames = {'suicidios_edad_10_14':'10-14 Años', 'suicidios_edad_15_19': '15-19 Años'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

#### Series de Tiempo

In [21]:
# Suicidios totales de menores de 19 años en Sonora
dff = sonora_data_tidy[sonora_data_tidy['sexo'] == 'Total']
dff = dff.groupby(['fecha','sexo']).suicidios.sum().reset_index()
fig = px.line(dff, x="fecha", y="suicidios", color='sexo',
                title='Numero de Suicidios de menores de 19 años en Sonora por Año',
                labels={'suicidios' : 'Numero de Suicidios',
                        'fecha': 'Año',
                        'sexo': ''})
fig.layout.update(showlegend=False) 
fig.show()


In [22]:
# Suicidios por Sexo
dff = sonora_data_tidy[sonora_data_tidy['sexo'] != 'Total']
dff = dff.groupby(['fecha','sexo']).suicidios.sum().reset_index()
fig = px.line(dff, x="fecha", y="suicidios", color='sexo',
                title='Numero de Suicidios de menores de 19 años en Sonora por Sexo',
                labels={'suicidios' : 'Numero de Suicidios',
                        'fecha': 'Año',
                        'sexo': 'Sexo'})
fig.show()


In [23]:
# Suicidios por Sexo
dff = sonora_data_tidy[sonora_data_tidy['sexo'] != 'Total']
dff = dff.groupby(['fecha','grupo_edad']).suicidios.sum().reset_index()
fig = px.line(dff, x="fecha", y="suicidios", color='grupo_edad',
                title='Numero de Suicidios de menores de 19 años en Sonora por Grupo de Edad (2010-2020)',
                labels={'suicidios' : 'Numero de Suicidios',
                        'fecha': 'Año',
                        'grupo_edad': 'Grupo de Edad'})
newnames = {'suicidios_edad_10_14':'10-14 Años', 'suicidios_edad_15_19': '15-19 Años'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

In [24]:
# Suicidios por Sexo
dff = sonora_data_tidy[sonora_data_tidy['sexo'] != 'Total']
#dff = dff.groupby(['fecha' 'grupo_ws','sexo']).suicidios.sum().reset_index()
fig = px.line(dff, x="fecha", y="suicidios", color='sexo', line_dash = 'grupo_edad',
                title='Numero de Suicidios de menores de 19 años en Sonora por Sexo y por Grupo de Edad',
                labels={'suicidios' : 'Numero de Suicidios',
                        'fecha': 'Año',
                        'sexo': 'Sexo',
                        'grupo_edad': 'Grupo de Edad'})
newnames = {'Hombres, suicidios_edad_10_14':'Hombres, 10-14 Años',
            'Hombres, suicidios_edad_15_19':'Hombres, 15-19 Años',
            'Mujeres, suicidios_edad_10_14':'Mujeres, 10-14 Años',
            'Mujeres, suicidios_edad_15_19':'Mujeres, 15-19 Años'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

#### Gráfico de barras apiladas

In [25]:
dff = sonora_data_tidy[sonora_data_tidy['sexo'] != 'Total']
fig = px.bar(dff, x="fecha", y="suicidios", color="grupo_edad",
             title="'Numero de Suicidios de menores de 19 años en Sonora por Grupo de Edad",
             labels={'suicidios' : 'Numero de Suicidios',
                        'fecha': 'Año',
                        'sexo': 'Sexo',
                        'grupo_edad': 'Grupo de Edad'})
newnames = {'suicidios_edad_10_14':'10-14 Años', 'suicidios_edad_15_19': '15-19 Años'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

fig.show()

In [26]:
dff = sonora_data_tidy[sonora_data_tidy['sexo'] != 'Total']
fig = px.bar(dff, x="fecha", y="suicidios", color="sexo",
             title="'Numero de Suicidios de menores de 19 años en Sonora por Sexo",
             labels={'suicidios' : 'Numero de Suicidios',
                        'fecha': 'Año',
                        'sexo': 'Sexo',
                        'grupo_edad': 'Grupo de Edad'})
fig.show()

#### Observaciones princpales

* En todos los años del 2010 al 2020, ha habido un mayor numero de suicidios en el grupo de edad de 15-19 años (con respecto al grupo de 10-14 años).
* La diferencia entre el número de suicidios por sexo, se ve mas acentuada en el grupo de 15-19 años, observandose un mayor número de suicidios en el grupo de los hombres.
* Del 2017 al 2018 casi se duplicó el número de suicidios (de 20 a 35), seguido por una disminución en el 2019 (de 35 a 26)
* Se recomienda hacer pruebas estadísticas para confirmar observaciones.


#### Guardando datos limpios

In [41]:
#Definimos el path donde los vamos a guardar
path_to_save = pathlib.Path('..', 'data_clean/salud_mental_clean.csv')

In [40]:
#Arreglamos los datos para poder pasarlos a csv
sonora_data_filtered = sonora_data_tidy[sonora_data_tidy['grupo_edad'] == 'suicidios_edad_10_14'].copy()
sonora_data_filtered = sonora_data_filtered.drop('grupo_edad', axis = 1)
sonora_data_filtered = sonora_data_filtered.groupby(['fecha', 'sexo']).sum()
sonora_data_filtered = sonora_data_filtered.unstack()
sonora_data_filtered.columns = sonora_data_filtered.columns.to_flat_index()
sonora_data_filtered.columns = ['suicidios_hombres', 'suicidios_mujeres','suicidios_totales']
sonora_data_filtered

,suicidios_hombres,suicidios_mujeres,suicidios_totales
fecha,,,
2010,0,6,6
2011,0,0,0
2012,0,1,1
2013,4,2,6
2014,2,2,4
2015,4,2,6
2016,0,2,2
2017,2,0,2
2018,7,1,8


In [42]:
sonora_data_filtered.to_csv(path_to_save)